#### Building a chatbot 

in this notrbook we'll go over an example of how to design and implement an llm-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

note that this chatbot that we build will only use the language model to have a conversation.There are several other related concepts that you may be looking for:

- conversational RAG: Enable a chatbot experience over an external source of data
- agent: build a chatbot that can take actions.


In [3]:
import os 
from dotenv import load_dotenv
from rich import print
load_dotenv(dotenv_path="../.env")

groq_api_key=os.getenv("GROQ_API_KEY")


In [4]:
from langchain_groq import ChatGroq

model=ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1093c4100>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x109395600>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage

print(model.invoke([HumanMessage(content="hi, my name is abhijit and i am an Ai engineer")]))

AIMessage(
    content="Hello Abhijit, it's nice to meet you!\n\nI'm glad to know you're an AI engineer. That's a fascinating 
field! \n\nWhat kind of AI projects are you working on? 🤖  I'm always eager to learn more about what people are 
doing in the world of AI.\n",
    additional_kwargs={},
    response_metadata={
        'token_usage': {
            'completion_tokens': 70,
            'prompt_tokens': 23,
            'total_tokens': 93,
            'completion_time': 0.127272727,
            'prompt_time': 0.001323968,
            'queue_time': 0.245229942,
            'total_time': 0.128596695
        },
        'model_name': 'gemma2-9b-it',
        'system_fingerprint': 'fp_10c08bf97d',
        'service_tier': 'on_demand',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--0d1c6b51-73ab-4608-8188-452f5b85bb7b-0',
    usage_metadata={'input_tokens': 23, 'output_tokens': 70, 'total_tokens': 93}
)

In [6]:
from langchain_core.messages import AIMessage

print(model.invoke([
    HumanMessage(content="hi, my name is abhijit and i am an Ai engineer"),
    AIMessage(content='''Hi Abhijit,\n\nNice to meet you! It's great to connect with another AI engineer.  \n\nWhat kind of 
work do you specialize in? I'm always interested to hear about the different projects and challenges people are 
tackling in the field.\n'''),
    HumanMessage(content="Hey What's my name and what do i do?")
]))

AIMessage(
    content="You said your name is Abhijit and that you are an AI engineer.  \n\nIs there anything else you'd like 
to tell me about yourself or your work?  I'm happy to chat! 😊 \n",
    additional_kwargs={},
    response_metadata={
        'token_usage': {
            'completion_tokens': 47,
            'prompt_tokens': 103,
            'total_tokens': 150,
            'completion_time': 0.085454545,
            'prompt_time': 0.002740607,
            'queue_time': 0.251745903,
            'total_time': 0.088195152
        },
        'model_name': 'gemma2-9b-it',
        'system_fingerprint': 'fp_10c08bf97d',
        'service_tier': 'on_demand',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--711e8c61-c6b9-4801-bbf2-4bbc75cbc1a1-0',
    usage_metadata={'input_tokens': 103, 'output_tokens': 47, 'total_tokens': 150}
)

#### Message History

We can use a Message History class to warp out model and make it standful.This will keep track of inputs and outputs of the model, and store them in some datastore.Future interactions will then load those messages and pass them into the chain as part of the input.lets see how to use this!

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [ ]:
config={"configurable":{"session_id":"chat1"}} ## creating a session id : chat1 

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="hi , my name is abhijit and i am a Ai engineer.")],
    config=config
)

In [13]:
print(response)

AIMessage(
    content="Hey Abhijit, that's great!  \n\nWhat are you working on these days as an AI engineer?  Anything 
exciting?  😄 \n",
    additional_kwargs={},
    response_metadata={
        'token_usage': {
            'completion_tokens': 33,
            'prompt_tokens': 105,
            'total_tokens': 138,
            'completion_time': 0.06,
            'prompt_time': 0.002811717,
            'queue_time': 0.250924322,
            'total_time': 0.062811717
        },
        'model_name': 'gemma2-9b-it',
        'system_fingerprint': 'fp_10c08bf97d',
        'service_tier': 'on_demand',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--d6710e60-0789-4abd-ad0f-81a6da0ebd42-0',
    usage_metadata={'input_tokens': 105, 'output_tokens': 33, 'total_tokens': 138}
)

In [14]:
print(with_message_history.invoke([HumanMessage(content="who am i ?")],config=config))

AIMessage(
    content="You are Abhijit, an AI engineer! 😊\n\nIs there anything else you'd like to tell me about yourself?  
What kind of AI work do you do?  \n",
    additional_kwargs={},
    response_metadata={
        'token_usage': {
            'completion_tokens': 39,
            'prompt_tokens': 150,
            'total_tokens': 189,
            'completion_time': 0.070909091,
            'prompt_time': 0.004383426,
            'queue_time': 0.250995513,
            'total_time': 0.075292517
        },
        'model_name': 'gemma2-9b-it',
        'system_fingerprint': 'fp_10c08bf97d',
        'service_tier': 'on_demand',
        'finish_reason': 'stop',
        'logprobs': None
    },
    id='run--42fdeec4-18f0-417e-b2e6-4f0449d8b2d8-0',
    usage_metadata={'input_tokens': 150, 'output_tokens': 39, 'total_tokens': 189}
)

In [17]:
print(store)

 ## all the context with llm

{
    'chat1': InMemoryChatMessageHistory(
        messages=[
            HumanMessage(
                content='hi , my name is abhijit and i am a Ai engineer.',
                additional_kwargs={},
                response_metadata={}
            ),
            AIMessage(
                content="Hi Abhijit, it's nice to meet you! That's awesome that you're an AI engineer. \n\nWhat 
kind of AI work do you do?  I'm always interested in learning more about what people are building in the field. 😊 
\n",
                additional_kwargs={},
                response_metadata={
                    'token_usage': {
                        'completion_tokens': 58,
                        'prompt_tokens': 24,
                        'total_tokens': 82,
                        'completion_time': 0.105454545,
                        'prompt_time': 0.001402138,
                        'queue_time': 0.251242062,
                        'total_time': 0.106856683
                    },
                    'model_name': 'gemma2-9b-it',
                    'system_fingerprint': 'fp_10c08bf97d',
                    'service_tier': 'on_demand',
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run--45f542d1-aced-4883-a52b-3d89ed35bf59-0',
                usage_metadata={'input_tokens': 24, 'output_tokens': 58, 'total_tokens': 82}
            ),
            HumanMessage(
                content='hi , my name is abhijit and i am a Ai engineer.',
                additional_kwargs={},
                response_metadata={}
            ),
            AIMessage(
                content="Hey Abhijit, that's great!  \n\nWhat are you working on these days as an AI engineer?  
Anything exciting?  😄 \n",
                additional_kwargs={},
                response_metadata={
                    'token_usage': {
                        'completion_tokens': 33,
                        'prompt_tokens': 105,
                        'total_tokens': 138,
                        'completion_time': 0.06,
                        'prompt_time': 0.002811717,
                        'queue_time': 0.250924322,
                        'total_time': 0.062811717
                    },
                    'model_name': 'gemma2-9b-it',
                    'system_fingerprint': 'fp_10c08bf97d',
                    'service_tier': 'on_demand',
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run--d6710e60-0789-4abd-ad0f-81a6da0ebd42-0',
                usage_metadata={'input_tokens': 105, 'output_tokens': 33, 'total_tokens': 138}
            ),
            HumanMessage(content='who am i ?', additional_kwargs={}, response_metadata={}),
            AIMessage(
                content="You are Abhijit, an AI engineer! 😊\n\nIs there anything else you'd like to tell me about 
yourself?  What kind of AI work do you do?  \n",
                additional_kwargs={},
                response_metadata={
                    'token_usage': {
                        'completion_tokens': 39,
                        'prompt_tokens': 150,
                        'total_tokens': 189,
                        'completion_time': 0.070909091,
                        'prompt_time': 0.004383426,
                        'queue_time': 0.250995513,
                        'total_time': 0.075292517
                    },
                    'model_name': 'gemma2-9b-it',
                    'system_fingerprint': 'fp_10c08bf97d',
                    'service_tier': 'on_demand',
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run--42fdeec4-18f0-417e-b2e6-4f0449d8b2d8-0',
                usage_metadata={'input_tokens': 150, 'output_tokens': 39, 'total_tokens': 189}
            )
        ]
    )
}